# Getting the data

The data is scraped from the wikipedia page:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

## Importing the libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import geocoder

## Create BS object

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

#print(soup.prettify()) # print the parsed data of html

## Scrape the data

In [3]:
table_contents=[] # list to save the data
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
table_contents[0:2]

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'}]

In [4]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df.head())

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government


# Getting coordinates

The coordinates for every neighboorhood are gotten.

In [5]:
# Load aux file to get latitude and longitude
coords = pd.read_csv('data/Geospatial_Coordinates.csv')

In [6]:
def get_lat_long(postal_code):
    # initialize variable to None
    lat_lng_coords = None
    # tries 3 times to get the coordinates through geocoder.
    count = 0
    while(lat_lng_coords is None and count<=3):
        count+=1
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    # if not, the aux file is used.
    if lat_lng_coords is None:
        # Get the row with the information
        row = coords[coords['Postal Code'] == postal_code]
        
        #Get the lat and long
        lat_lng_coords = (row['Latitude'].iat[0], row['Longitude'].iat[0])

    return lat_lng_coords

In [7]:
lat_longs = [get_lat_long(code) for code in df['PostalCode']]
df['latitude'] = [x[0] for x in lat_longs]
df['longitude'] = [x[1] for x in lat_longs]

In [8]:
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
